# WIP: Books Crossing Dataset

Books crossing data set contains ratings for books by users. This data set contains three files:
    - BX-Books.csv: List of books along with meta data
    - BX-Book-Ratings.csv: The ratings for all the books at a user level
    - BX-Users.csv: Meta data on the users

Data can be found [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

This notebook aims to build a recommendation engine to recommend to new books to users and is based of [this](https://towardsdatascience.com/how-did-we-build-book-recommender-systems-in-an-hour-the-fundamentals-dfee054f978e) post, with some improvements.

## Reading in the Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

When reading in the data for books, there may be lines where there are more columns than present in the header (Each book also has image URLs linking the cover of the book, a column may have more than one URL), this will throw an error. Setting `error_bad_lines` to false will skip these lines and will not be part of the data frame. 

In [7]:
# Read in the data
books = pd.read_csv('..\Data\BX-Books.csv', sep=';', header=0, encoding="latin-1", error_bad_lines=False) # sorry, I'm on Windows :(

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
c:\users\harpal\envs\datascience\lib\site-packages\IPython\core\interactiv

Looks like only 4 lines were skipped, this should not greatly impact the model

In [10]:
ratings = pd.read_csv('..\Data\BX-Book-Ratings.csv', sep=';', header=0, encoding="latin-1")

In [12]:
users = pd.read_csv('..\Data\BX-Users.csv', sep=';', encoding="latin-1")

In [13]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Let's see what the data looks like for our datasets

In [17]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0743442482,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/088404632X.0...,http://images.amazon.com/images/P/076791340X.0...,http://images.amazon.com/images/P/067976402X.0...
freq,1,27,632,13903,7535,2,2,2


In [22]:
books[books['Book-Author']=='Agatha Christie'].shape

(632, 8)

In [23]:
books.shape

(271360, 8)

In [21]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [18]:
ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [20]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [19]:
users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000
